In [ ]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import xml

In [ ]:
INDICES=['V', 'G', 'A', 'R', 'RS', 'AG', 'AM', 'ES', 'FV']

# page_votes = "https://www.fantagazzetta.com/voti-serie-a/2016-17/"
# page_votes = "http://www.gazzetta.it/calcio/fantanews/voti/serie-a-2016-17/giornata-"
pages_grades = {'2014':"http://www.gazzetta.it/calcio/fantanews/voti/serie-a-2014-15/giornata-",
                '2015':"http://www.gazzetta.it/calcio/fantanews/voti/serie-a-2015-16/giornata-",
                '2016':"http://www.gazzetta.it/calcio/fantanews/voti/serie-a-2016-17/giornata-",}
n_giornate = 38



folder = "C:\\Users\\zuk-8\\Programs\\workspace\\jupiter_notebooks\\Andrea"
fn = "fanta_grades.csv"

In [ ]:
def clean_table_cells(str):
    str = str.replace('\r', '').replace('\n', '').replace('\t', '').replace(' ', '')
    try:
        return float(str)
    except ValueError:
        return pd.np.NaN

def get_team_soups(page_votes, giornata):
    # Get html data from page+giornata
    page = page_votes+str(giornata)
    r = requests.get(page)
    data = r.text
    # Navigate inside data using BS
    soup = BeautifulSoup(data, "lxml")
    soup1 = [sec for sec in soup.html.body.find_all('section') if sec.get('class', None)==['main-container']][0]
    soup2 = [sec for sec in soup1.find_all('section') if sec.get('class', None)==['section-standard-row']][0]
    soup3 = [sec for sec in soup2.find_all('div') if sec.get('class', None)==["MXXX-central-articles-main-column"]][0]
    soup4 = [sec for sec in soup3.find_all('div') if
                   sec.get('class', None)==['magicDayList', 'listView', 'magicDayListChkDay']][0]
    return [sec for sec in soup4.find_all('div') if sec.get('class', None)==['singleRound']]

def get_player_soup(team_soup):
    team_list = [sec for sec in team_soup.find_all('div') if sec.get('class', None)==["magicTeamListContainer"]][0]
    team_grades = [sec for sec in team_list.find_all('ul') if sec.get('class')==["magicTeamList"]][0]
    team = [span.contents[0] for span in team_grades.li.div.find_all('span') if span.get('class', None)==["teamNameIn"]][0]
    player_soups = team_grades.find_all('li')[1:]
    return team, player_soups

def get_player_stats(player_soup, indices=INDICES):
    player_name = [span.string for span in player_soup.div.div.find_all('span') if span.get('class',None)==["playerNameIn"]][0]
    role = [span.string for span in player_soup.div.find_all('span') if
            span.get('class',None)==["playerRole", "show-for-small"]][0]
    stats = pd.Series(data=[clean_table_cells(div.string)
            for div in player_soup.find_all('div') if div.get('class', None)[0] == "inParameter"],
      index=indices)
    player_in = [sp for sp in player_soup.find_all('span') if sp.get('class',None)==['playerStats', 'icon', 'down']]
    player_out = [sp for sp in player_soup.find_all('span') if sp.get('class',None)==['playerStats', 'icon', 'up']]
    pi = 1 if len(player_in)>0 else 0
    po = 1 if len(player_out)>0 else 0
    stats = stats.append(pd.Series(index=['In', 'Out'], data = [pi, po]))
    return player_name, role, stats

In [ ]:
# all_stats = pd.DataFrame()
# for giornata in range(1,n_giornate+1):
#     team_stats = pd.DataFrame()
#     for team_soup in get_team_soups(page_votes, giornata):
#         team, player_soups = get_player_soup(team_soup)
#         player_stats = pd.DataFrame()
#         for player_soup in player_soups:
#             player_name, role, stats = get_player_stats(player_soup, indices=INDICES)
#             ps = pd.DataFrame({player_name:stats}).T
#             ps['Role'] = role
#             player_stats = pd.concat([player_stats, ps])
#         player_stats = player_stats.reset_index().rename(columns={'index':'Player'})
#         player_stats['Team'] = team
#         team_stats = pd.concat([team_stats, player_stats])
#     team_stats['Giornata'] = str(giornata)
#     all_stats = pd.concat([all_stats, team_stats])
# all_stats['Season'] = '2016'
# all_stats = all_stats.set_index(['Season', 'Giornata', 'Team', 'Player'])
# all_stats.to_csv(os.path.join(folder, fn))

In [ ]:
all_stats = pd.DataFrame()
for season,page_grades in pages_grades.items():
    season_stats = pd.DataFrame()
    for giornata in range(1,n_giornate+1):
        team_stats = pd.DataFrame()
        for team_soup in get_team_soups(page_grades, giornata):
            team, player_soups = get_player_soup(team_soup)
            player_stats = pd.DataFrame()
            for player_soup in player_soups:
                player_name, role, stats = get_player_stats(player_soup, indices=INDICES)
                ps = pd.DataFrame({player_name:stats}).T
                ps['Role'] = role
                player_stats = pd.concat([player_stats, ps])
            player_stats = player_stats.reset_index().rename(columns={'index':'Player'})
            player_stats['Team'] = team
            team_stats = pd.concat([team_stats, player_stats])
        team_stats['Giornata'] = str(giornata)
        season_stats = pd.concat([season_stats, team_stats])
    season_stats['Season'] = season
    all_stats = pd.concat([all_stats, season_stats])
all_stats = all_stats.set_index(['Season', 'Giornata', 'Team', 'Player'])
all_stats.to_csv(os.path.join(folder, fn))

In [ ]:
# # Get page links

# page = "//www.fantagazzetta.com/squadre"
# player_teams = {}
# player_links = {}

# teams = ['Atalanta', 'Bologna', 'Cagliari', 'Chievo', 'Crotone', 'Empoli', 'Fiorentina',
#          'Genoa', 'Inter', 'Juventus', 'Lazio', 'Milan', 'Napoli', 'Palermo', 'Pescara',
#          'Roma', 'Sampdoria', 'Sassuolo', 'Torino', 'Udinese']

# for team in teams:
#     site = page+"/"+team
#     r  = requests.get("https:"+site)
#     data = r.text
#     soup = BeautifulSoup(data, "lxml")
#     for link in soup.find_all('a'):
#         l = link.get('href', '')
#         if l.startswith(site):
#             player_teams[link.contents[0]] = team
#             player_links[link.contents[0]] = l

# for player,link in list(player_links.items())[1:2]:
#     site = link + "/2/2016-17"
    
#     r  = requests.get("https:"+site)
#     data = r.text
#     soup = BeautifulSoup(data, "html.parser")